# train

In [ ]:
%maven ai.djl:api:0.6.0
%maven ai.djl:basicdataset:0.6.0
%maven ai.djl:model-zoo:0.6.0
%maven ai.djl.mxnet:mxnet-engine:0.6.0
%maven org.slf4j:slf4j-api:1.7.26
%maven org.slf4j:slf4j-simple:1.7.26
%maven net.java.dev.jna:jna:5.3.0
        
// See https://github.com/awslabs/djl/blob/master/mxnet/mxnet-engine/README.md
// for more MXNet library selection options
%maven ai.djl.mxnet:mxnet-native-auto:1.7.0-b

In [2]:

import java.nio.file.*;

import ai.djl.*;
import ai.djl.basicdataset.*;
import ai.djl.ndarray.types.*;
import ai.djl.training.*;
import ai.djl.training.dataset.*;
import ai.djl.training.initializer.*;
import ai.djl.training.loss.*;
import ai.djl.training.listener.*;
import ai.djl.training.evaluator.*;
import ai.djl.training.optimizer.*;
import ai.djl.training.util.*;
import ai.djl.basicmodelzoo.cv.classification.*;
import ai.djl.basicmodelzoo.basic.*;

In [3]:
int batchSize = 32;
Mnist mnist = Mnist.builder().setSampling(batchSize, true).build();
mnist.prepare(new ProgressBar());

[IJava-executor-0] INFO ai.djl.mxnet.jna.LibUtils - Downloading libmxnet.dylib ...


Downloading: 100% |████████████████████████████████████████|


In [5]:
Model model = Model.newInstance("mlp");
model.setBlock(new Mlp(28 * 28, 10, new int[] {128, 64}));

In [6]:
DefaultTrainingConfig config = new DefaultTrainingConfig(Loss.softmaxCrossEntropyLoss())
    //softmaxCrossEntropyLoss is a standard loss for classification problems
    .addEvaluator(new Accuracy()) // Use accuracy so we humans can understand how accurate the model is
    .addTrainingListeners(TrainingListener.Defaults.logging());

// Now that we have our training configuration, we should create a new trainer for our model
Trainer trainer = model.newTrainer(config);

[IJava-executor-0] INFO ai.djl.training.listener.LoggingTrainingListener - Training on: cpu().
[IJava-executor-0] INFO ai.djl.training.listener.LoggingTrainingListener - Load MXNet Engine Version 1.7.0 in 0.107 ms.


In [7]:
trainer.initialize(new Shape(1, 28 * 28));

In [9]:
int epoch = 2;

for (int i = 0; i < epoch; ++i) {
    int index = 0;
    
    // We iterate through the dataset once during this epoch
    for (Batch batch : trainer.iterateDataset(mnist)) {
        
        // During trainBatch, we update the loss and evaluators with the results for the training batch.
        EasyTrain.trainBatch(trainer, batch);
        
        // Now, we update the model parameters based on the results of the latest trainBatch
        trainer.step();
        
        // We must make sure to close the batch to ensure all the memory associated with the batch is cleared quickly.
        // If the memory isn't closed after each batch, you will very quickly run out of memory on your GPU
        batch.close();
    }
    // reset training and validation evaluators at end of epoch
    trainer.notifyListeners(listener -> listener.onEpoch(trainer));
}

Training:    100% |████████████████████████████████████████| 


[IJava-executor-0] INFO ai.djl.training.listener.LoggingTrainingListener - Epoch 1 finished.


Training:    100% |████████████████████████████████████████| 


[IJava-executor-0] INFO ai.djl.training.listener.LoggingTrainingListener - Epoch 2 finished.


In [10]:

Path modelDir = Paths.get("build/mlp");
Files.createDirectories(modelDir);

model.setProperty("Epoch", String.valueOf(epoch));

model.save(modelDir, "mlp");

model

Model (
	Name: mlp
	Model location: /Users/zhoubin/PycharmProjects/testsh/build/mlp
	Data Type: float32
	Epoch: 2
)

# infer

In [2]:
%maven ai.djl:api:0.6.0
%maven ai.djl:model-zoo:0.6.0
%maven ai.djl.mxnet:mxnet-engine:0.6.0
%maven ai.djl.mxnet:mxnet-model-zoo:0.6.0
%maven org.slf4j:slf4j-api:1.7.26
%maven org.slf4j:slf4j-simple:1.7.26
%maven net.java.dev.jna:jna:5.3.0
    
// See https://github.com/awslabs/djl/blob/master/mxnet/mxnet-engine/README.md
// for more MXNet library selection options
%maven ai.djl.mxnet:mxnet-native-auto:1.7.0-b

In [3]:
import java.awt.image.*;
import java.nio.file.*;
import java.util.*;
import java.util.stream.*;
import ai.djl.*;
import ai.djl.basicmodelzoo.basic.*;
import ai.djl.ndarray.*;
import ai.djl.modality.*;
import ai.djl.modality.cv.*;
import ai.djl.modality.cv.util.NDImageUtils;
import ai.djl.translate.*;

In [4]:
var img = ImageFactory.getInstance().fromUrl("https://djl-ai.s3.amazonaws.com/resources/images/0.png");
img.getWrappedImage();

In [11]:
Path modelDir = Paths.get("build/mlp");
Model model = Model.newInstance("mlp");
model.setBlock(new Mlp(28 * 28, 10, new int[] {128, 64}));
model.load(modelDir);

In [15]:
Translator<Image, Classifications> translator = new Translator<Image, Classifications>() {

    @Override
    public NDList processInput(TranslatorContext ctx, Image input) {
        // Convert Image to NDArray
        NDArray array = input.toNDArray(ctx.getNDManager(), Image.Flag.GRAYSCALE);
        return new NDList(NDImageUtils.toTensor(array));
    }

    @Override
    public Classifications processOutput(TranslatorContext ctx, NDList list) {
        NDArray probabilities = list.singletonOrThrow().softmax(0);
        List<String> indices = IntStream.range(0, 10).mapToObj(String::valueOf).collect(Collectors.toList());
        return new Classifications(indices, probabilities);
    }
    
    @Override
    public Batchifier getBatchifier() {
        return Batchifier.STACK;
    }
};

In [17]:
var predictor = model.newPredictor(translator);

In [19]:
var classifications = predictor.predict(img);

In [20]:
classifications

[
	class: "0", probability: 0.99990
	class: "2", probability: 0.00008
	class: "6", probability: 2.0e-06
	class: "8", probability: 9.0e-07
	class: "1", probability: 3.5e-07
]